Deep Learning
=============

Assignment 4 and Solutions
------------

Previously in [2_fullyconnected.ipynb](./assg2_fullyconnected_sol.ipynb) and [3_regularization.ipynb](./3_regularization.ipynb), we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1  # Grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)).astype(np.float32)
    labels = (np.arange(NUM_LABELS) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

In [6]:
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden1 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, NUM_CHANNELS, depth], stddev=0.1))
    biases_hidden1 = tf.Variable(tf.zeros([depth]))
    weights_hidden2 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_hidden2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    weights_hidden3 = tf.Variable(
        tf.truncated_normal(
            [IMAGE_SIZE // (2 * 2) * IMAGE_SIZE // (2 * 2) * depth, num_hidden], stddev=0.1))
    biases_hidden3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    weights_output = tf.Variable(
        tf.truncated_normal([num_hidden, NUM_LABELS], stddev=0.1))
    biases_output = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
  
    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, weights_hidden1, [1, 2, 2, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + biases_hidden1)
        conv2 = tf.nn.conv2d(hidden1, weights_hidden2, [1, 2, 2, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + biases_hidden2)
        shape2 = hidden2.get_shape().as_list()
        reshape2 = tf.reshape(hidden2, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape2, weights_hidden3) + biases_hidden3)
        return tf.matmul(hidden3, weights_output) + biases_output
  
    # Training computation.
    score = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.867388
Minibatch accuracy: 25.0%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.946463
Minibatch accuracy: 25.0%
Validation accuracy: 31.1%
Minibatch loss at step 100: 1.133244
Minibatch accuracy: 50.0%
Validation accuracy: 62.2%
Minibatch loss at step 150: 0.531447
Minibatch accuracy: 87.5%
Validation accuracy: 74.9%
Minibatch loss at step 200: 0.953384
Minibatch accuracy: 75.0%
Validation accuracy: 74.6%
Minibatch loss at step 250: 1.171133
Minibatch accuracy: 68.8%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.319777
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.586691
Minibatch accuracy: 87.5%
Validation accuracy: 75.6%
Minibatch loss at step 400: 0.349005
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.832078
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 500: 0.635169
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

In [17]:
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1  # Grayscale
batch_size = 16
patch_size = 5
depth = 16
sd = 0.1
cv_stride = 1
mp_kernel = 2
mp_stride = 2
num_hidden = 64
num_steps = 1001

In [18]:
graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_x = tf.placeholder(
        tf.float32, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    tf_train_y_ = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_x = tf.constant(valid_dataset)
    tf_test_x = tf.constant(test_dataset)
  
    # Variables.
    W_hidden1 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, NUM_CHANNELS, depth], stddev=sd))
    b_hidden1 = tf.Variable(tf.zeros([depth]))
    W_hidden2 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=sd))
    b_hidden2 = tf.Variable(tf.constant(1.0, shape=[depth]))
    W_hidden3 = tf.Variable(
        tf.truncated_normal(
            [IMAGE_SIZE // (mp_kernel * mp_stride) * IMAGE_SIZE // (mp_kernel * mp_stride) * depth, num_hidden], 
            stddev=sd))
    b_hidden3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    W_output = tf.Variable(
        tf.truncated_normal([num_hidden, NUM_LABELS], stddev=0.1))
    b_output = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
  
    # CNN in vanilla version:
    # Our convolutions uses a stride of one and are zero padded so that 
    # the output is the same size as the input. 
    # Our pooling is plain old max pooling over 2x2 blocks.
    def conv2d(x, W, cv_stride=2):
        return tf.nn.conv2d(x, W, strides=[1, cv_stride, cv_stride, 1], padding='SAME')
    
    def max_pool2d(x, mp_kernel=2, mp_stride=2):
        return tf.nn.max_pool(
            x, ksize=[1, mp_kernel, mp_kernel, 1], strides=[1, mp_stride, mp_stride, 1], padding='SAME')
    
    def model(data):
        hidden1 = tf.nn.relu(conv2d(data, W_hidden1, cv_stride) + b_hidden1)
        hidden1_pool = max_pool2d(hidden1, mp_kernel, mp_stride)
        hidden2 = tf.nn.relu(conv2d(hidden1_pool, W_hidden2, cv_stride) + b_hidden2)
        hidden2_pool = max_pool2d(hidden2, mp_kernel, mp_stride)
        shape2 = hidden2_pool.get_shape().as_list()
        reshape2 = tf.reshape(hidden2_pool, [shape2[0], shape2[1] * shape2[2] * shape2[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape2, W_hidden3) + b_hidden3)
        return tf.matmul(hidden3, W_output) + b_output
  
    # Training computation.
    score = model(tf_train_x)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(score, tf_train_y_))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(score)
    valid_prediction = tf.nn.softmax(model(tf_valid_x))
    test_prediction = tf.nn.softmax(model(tf_test_x))

In [19]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_x: batch_data, tf_train_y_: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.377515
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.860863
Minibatch accuracy: 31.2%
Validation accuracy: 35.9%
Minibatch loss at step 100: 0.999200
Minibatch accuracy: 75.0%
Validation accuracy: 62.9%
Minibatch loss at step 150: 0.370988
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%
Minibatch loss at step 200: 1.086647
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.226613
Minibatch accuracy: 62.5%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.321869
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.605127
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.263287
Minibatch accuracy: 93.8%
Validation accuracy: 80.4%
Minibatch loss at step 450: 0.826651
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.461563
Minibatch accuracy: 93.8%
Validation accuracy: 81.8%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

TODO: Propose a convolutional net referred to LeNet5 architecture ([LeCun, Bottou, Bengio & Haffner (Proc IEEE, 1998)](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf))
- Layer 0: Input with 1@28x28
- Layer 1: Convolutions with 6@28x28
- 